In [1]:
import requests
import re
import os
import PyPDF2
from io import BytesIO
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urlparse
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from collections import Counter, defaultdict

In [2]:
def sanitize_filename(filename):
    # List of invalid characters for file names
    invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
    for char in invalid_chars:
        filename = filename.replace(char, '_')  # Replace invalid characters with underscore
    return filename

In [3]:
url = "https://www.ntnu.no/cerg/media"

In [16]:
articles = []

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

def is_readable_article(tag):
    return tag.name == 'a' and tag.get_text(strip=True) == 'Les saken'

a_tags = soup.find_all(is_readable_article)

for a_tag in a_tags:
    link = a_tag['href']
    
    article = {
        'link': link,
    }

    articles.append(article)
len(articles)

186

### Filter out bad domains

In [15]:
excluded_domains = ["dagbladet.no", "kk.no", "vi.no", "fosna-folket.no"] # paywall
articles = [article for article in articles if not any(domain in article['link'] for domain in excluded_domains)]
len(articles)

166

In [11]:
import os
import subprocess

failed_links = []
timeout_links = []

# Create directory for storing PDFs if it doesn't exist
os.makedirs('../data/articles', exist_ok=True)

def save_article_as_pdf(link, filename):
    command = [
        'C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe',
        link,
        filename
    ]
    try:
        subprocess.run(command, timeout=10, check=True)
    except subprocess.TimeoutExpired:
        timeout_links.append(link)
        print(f'Timeout expired for {link}.')
    except subprocess.CalledProcessError as e:
        failed_links.append(link)
        print(f'Failed to save {link} as PDF: {e}')

for article in articles:
    # Construct a filename based on the article link
    filename = '../data/articles/' + article['link'].split('/')[-1] + '.pdf'
    save_article_as_pdf(article['link'], filename)



Failed to save https://forskning.no/hjernen-hjerteinfarkt-hjertet/tre-usunne-trekk-gir-mye-hoyere-risiko-for-hjerteinfarkt/2244618 as PDF: Command '['C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe', 'https://forskning.no/hjernen-hjerteinfarkt-hjertet/tre-usunne-trekk-gir-mye-hoyere-risiko-for-hjerteinfarkt/2244618', '../data/articles/2244618.pdf']' returned non-zero exit status 1.
Timeout expired for https://www.nrk.no/trondelag/rotter-med-alzheimer-sykdom-fikk-nye-hjerneceller-etter-a-ha-fatt-blod-fra-trente-menn-1.16530238.
Timeout expired for https://www.nrk.no/trondelag/johannes-nonas-langhelle-og-ole-petter-hjelle-motte-veggen-_-fysisk-aktivitet-ble-veien-tilbake-1.16501151.
Failed to save https://forskning.no/genetikk-trening/sliter-du-med-a-sla-den-personlige-rekorden-genene-har-trolig-noe-av-skylda/2231256 as PDF: Command '['C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe', 'https://forskning.no/genetikk-trening/sliter-du-med-a-sla-den-personlige-rekorden-genene-har-trolig-n

In [17]:
print(len(articles), "successes.", len(failed_links), "failures.", len(timeout_links), "timeouts.")

186 successes. 56 failures. 36 timeouts.
